# SQL con Python: JOIN 
Juntan datos de varias tablas, **necesitas tener uno o varios campos comunes entre ambas tablas, que se denominan CLAVES**
Se usan por ejemplo, si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos que acudir a otras tablas donde esté ese identificador de cliente y aplicar un `JOIN`.
 Es lo que se conoce como *pegar campos* a otra tabla. 
 
 si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos 
  Si quieres filtrar dentro de tu tabla total de pedidos los que no se llegaron a entregar, podrías aplicar lo que se llama un `INNER JOIN` de manera que te quedes con lo común en ambas tablas


### Tipos de JOINS
Hay que saber que tipo hay que poner.
Para unir dos tablas usamos:


### Left (Outer) Join
 este JOIN en el que manda la tabla de la IZQUIERDA o primera tabla mencioanda se le llama LEFT JOIN
 y también LEFT OUTTER JOIN.

Su sintaxis en SLQ:
```SQL
SELECT A.id_empleado, A.nombre, A.id_departamento, B.nombre_departamento 
FROM Empleados AS A
LEFT JOIN Departamentos AS B #le decimos que joint queremos
ON A.id_departamento = B.id_departamento
```
A, pertenece a la tabla empleados y B a la de departamentos, que son las tablas que queremos unir.
Es un left porque viene detrás del front.



### Right (outer) join

De igual manera, aunque quizás más contraintuitivo, podemos pegar a cada departamento los datos de los empleados que tengan igual id_departamento y NaN si no hay empleados en ese departamento.

Su sintáxis.

```SQL
SELECT A.id_empleado, A.nombre, A.id_departamento, B.nombre_departamento
FROM Empleados AS A
RIGHT JOIN Departamentos AS B
ON A.id_departamento = B.id_departamento
```
La misma sintaxis, solo que cambiamos el right por el left

### Inner Join

```SQL 
SELECT A.id_empleado,A.nombre,A.id_departamento, B.nombre_departamento 
FROM Empleados AS A
INNER JOIN Departamentos AS B
ON A.id_departamento = B.id_departamento
```
Es un cruce, si hay valores sin nintersección, no los une, en este campo no está recursos humanos ni luis

### Full Outer Join

Une todos los campos aunque sean nulos
```SQL 
SELECT A.id_empleado,A.nombre,A.id_departamento, B.nombre_departamento 
FROM Empleados AS A
FULL OUTER JOIN Departamentos AS B
ON A.id_departamento = B.id_departamento
```

Existen más joints

# SQL con Python: JOIN Ejemplos (I)

In [ ]:
#ejecutamos:
import pandas as pd
import sqlite3

# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("./data/chinook_joins.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor_bootcamp = connection.cursor()

In [ ]:
#la funciónm que vamos a usar
def sql_query(query):
    cursor_bootcamp.execute(query)
    if query.lower().strip().startswith("select "):
        ans = cursor_bootcamp.fetchall()
        names = [description[0] for description in cursor_bootcamp.description]
        return pd.DataFrame(ans,columns=names)
    return None

### Left Join (I)

es el JOIN en el que manda la tabla de la "izquierda"
ponemos en el `FROM`
Es la tabla de la qu
 queremos mantener todas las filas y pegar
le campos de otra tabla aunque no haya cruce, en cuyo caso se rellenarán con nulos (NULL o NaN si lo pasamos a Pandas)

In [ ]:
# Uno bastante claro: Queremos pegar a las canciones el nombre del album al que pertenecen
# Manda la tabla de canciones (tracks) 
# sobre el de álbumes (albums)
# Si te fijas en el modelo de datos la columna o clave de cruce es AlbumId (Foreing_Key en tracks, Primary_Key en albums)
query = '''
SELECT a.*, b.title as album_title    -- Nos quedamos con todos los campos de a, esto es muy típico y sólo con el de b que nos interese
FROM tracks AS a        -- track la ponemos la primera, es la de la "izquierda", y como queremos que mande (todas sus filas) entonces es un LEFT JOIN
LEFT JOIN albums AS b      -- album está detrás del JOIN, es la de la "derecha"
ON a.albumid = b.albumid
'''

df_lj = sql_query(query)
df_lj

método `info`, el campo que hemos llamado contiene nulos.
algunos de los valores de `albumid` de la tabla tracks (la de la "izquierda") no están presentes en la tabla albums y por eso rellena ese campo con NaN. 
 LEFT join manda la tabla de la izquierda, es decir se mantienen todos los registros de la izquierda (con los campos que hayamos seleccionado, en este caso todos) y si no encuentra correspondencia en la derecha rellena con nulos.

In [ ]:
#obtenemos los valores de album id que no tienen correspondencia en la tabla album
faltan = df_lj.loc[df_lj.album_title.isna(),"AlbumId"].to_list() # SQL no es case-sensitive pero Pandas sí, por eso AlbumId
#df, aquellas filas donde el albun tittle es nan, devuelve los que contenían la otra tabla

In [ ]:
faltan = list(set(faltan))

# Lo convertimos a una lista de strings (porque lo vamos a necesitar a continuación)
faltan = [str(elemento) for elemento in faltan]
#se continua haciendo la conversaión a set y luego a list y luego esos valores se pasan a string

In [ ]:
#Se construye una query en la cual esta lista, se convierte en un texto separado por comas, para poder meterlo en la query que habíamos visto
query = f'''
SELECT title 
FROM albums
WHERE albumid in ({",".join(faltan)}) -- el ",".join(faltan) devuelve un string con los ids separada por comas, que es lo que espera SQL
'''
#metemos los títulos de la tabla albums cuyos id estén en esa lista
sql_query(query)
#nos devuelve un df vacío

### Left Join (II)


In [ ]:
query = '''
SELECT *
FROM invoice_items
'''
df_invoices = sql_query(query)
df_invoices
#queremos pegar la canción en invoice items, pegando un campo de la otra tabla

In [ ]:
query = '''
SELECT a.*, tracks.name 
FROM Invoice_Items AS a
LEFT JOIN tracks
ON a.TrackId = tracks.TrackId
'''

df_invoices_with_names = sql_query(query)

In [ ]:
df_invoices_with_names

saber el top de ventas y sus ventas:

In [ ]:
df_invoices_with_names.Name.value_counts().nlargest(3)

### Right Join

manda la tabla de la "derecha", la que ponemos detrás del JOIN. Es la tabla de la que queremos mantener todas las filas y pegarle campos de otra tabla aunque no haya cruce, en cuyo caso se rellenarán con nulos (NULL o NaN si lo pasamos a Pandas)

In [61]:
# Hagamos el inverso del anterior pero por la derecha: Queremos pegar a cada disco las canciones
# Manda la tabla de álbumes (albums)  
# sobre el de canciones (tracks)
# Como en la sesión anterior la clave de cruce es AlbumId (Foreing_Key en tracks, Primary_Key en albums)
query = '''
SELECT b.*, a.name   --Selecciona todos los campos de albums y los saca de tracks  
FROM tracks AS a
RIGHT JOIN albums AS b
ON a.albumid = b.albumid
'''
df_rj = sql_query(query)
df_rj #me da error

NameError: name 'sql_query' is not defined

In [ ]:
query = '''
SELECT *
FROM albums
'''
df_albums = sql_query(query)
df_albums.info() #se amplia por el número de canciones, graciasd al join

In [ ]:
print(df_albums.AlbumId.is_unique) #nos da info del índice
print(df_rj.AlbumId.Is_unique) #da false

### Inner Join

sólo quiero quedarme con las filas que tengan valores comunes en las dos tablas

In [ ]:
# Vamos a quedarnos ahora con las canciones para las cuales si haya disco
query = '''
SELECT tracks.*, albums.title    --no hace falta usar siempre enl nombre de los alias, podemos poner el nombre de las tablas
FROM tracks
INNER JOIN albums
ON tracks.AlbumId = albums.AlbumId
'''
df_inner = sql_query(query)
df_inner.info() #no hay más  nulos de los que habían
#se ha reducido el número de salidas en co0mparación al anterior.


### Full join

se queda todos los cruces posibles (por valores existentes) aunque haya nulo. Sqlite3 no admite Full Join por lo que hay que simularlo.

In [56]:
#FULL JOIN
#FULL JOIN isn't supported in sqlite, 
#so we use a LEFT JOIN + RIGHT JOIN(inverse LEFT JOIN) as a workaround

query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid
UNION ALL -- usa el union all que permite unir  la salida de dos queries por separado, no es una subquery una dentro de otra.
SELECT *
FROM tracks AS a
LEFT JOIN invoice_items AS b
ON a.trackid =b.trackid;
'''

sql_query(query)

NameError: name 'sql_query' is not defined

# SQL en Python: Gestion BD (I)

### Creación de Bases de Datos y tablas

In [57]:
import pandas as pd
import sqlite3

In [58]:
#conexión a base de datos que no existe, pero la crea
connection = sqlite3.connect("base_de_datos_I.db") 

crear el cursor para poder interactuar con ella:

In [59]:
cursor_gestion = connection.cursor()

Y ahora ya podemos crear una tabla siguiendo la siguiente sintaxis:

```SQL
CREATE TABLE nombre_tabla (
    columna1 tipo_de_dato restricciones, --creamos una linea donde ponemos el tipo(int, float, date... y restricciones(si va  ser primary key,
    columna2 tipo_de_dato restricciones, --si no puede ser nulo...)
    ...                                  --
    columnaN tipo_de_dato restricciones
);
```

- **nombre_tabla:** Es el nombre de la tabla que queremos crear.
- **columna1, columna2, ..., columnaN:** Son los nombres de las columnas de la tabla.
- **tipo_de_dato:** Es el tipo de dato de cada columna (por ejemplo, `INT` para enteros, `VARCHAR` o `CHAR` para cadenas de texto, `DATE` para fechas, etc.).
- **restricciones:** Son las restricciones o reglas para cada columna (opcional). Algunas restricciones comunes incluyen `PRIMARY KEY`, `NOT NULL`, `UNIQUE`, `FOREIGN KEY`, etc. (que corresponden a un índice único, a que no puede dejarse vacío el campo, a que no puede repetirse o a que tendrá que enlazarse con la clave en otra tabla)

In [60]:
query_create = '''
CREATE TABLE Master_Class ( --El nombre
ID       INT PRIMARY KEY, -- nombre tipo_de_dato restricción como hemos visto antes tipo entero
NOMBRE   TEXT NOT NULL, --texto
EDAD     INT NOT NULL,
CIUDAD   CHARS(50), -- Le decimos que este campo siempre tiene 50 caracteres, y los que no llene los rellenará con espacios
NOTAS    FLOAT
)
'''
cursor_gestion.execute(query_create)

OperationalError: table Master_Class already exists

Y una vez la tenemos, podemos buscarla en la tabla maestra:

In [44]:
query = "SELECT * FROM sqlite_master WHERE type =='table'" #buscamos en la tabla maestra
cursor_gestion.execute(query) #La ejecutamos
cursor_gestion.fetchall() #el fetchall, recupera todas las filas que ha traido la ejecución

[('table',
  'Master_Class',
  'Master_Class',
  2,
  'CREATE TABLE Master_Class ( --El nombre\nID       INT PRIMARY KEY, -- nombre tipo_de_dato restricción como hemos visto antes tipo entero\nNOMBRE   TEXT NOT NULL, --texto\nEDAD     INT NOT NULL,\nCIUDAD   CHARS(50), -- Le decimos que este campo siempre tiene 50 caracteres, y los que no llene los rellenará con espacios\nNOTAS    FLOAT\n)')]

En las tablas maestras lo que se guardan no son tanto los campos como los campos creados

### Insert
En la tabla creadaa podemos ingestar registros (nombre técnico), modificarlos y borrarlos

 sintaxis de un insert:

```SQL
INSERT INTO nombre_tabla (columna1, columna2, columna3, ...)
VALUES (valor1, valor2, valor3, ...);
```
- **nombre_tabla:** Nombre de la tabla donde se insertarán los datos.
- **columna1, columna2, ... :** Las columnas de la tabla en las que se insertarán los datos. No es necesario incluir todas las columnas, especialmente si algunas tienen valores predeterminados o son autoincrementables.
- **valor1, valor2, ... :** Los valores correspondientes a las columnas especificadas. Deben estar en el mismo orden que las columnas y deben ser del tipo de dato adecuado para cada columna.


Insertemos unos cuantos valores:

- Luis, 24, Madrid, 8.5
- Ana, 32, Lugo, 6.25
- Juan, 35, Bilbao, 5.55
- Nuria, 41, Alicante, 9.75

Se pueden hacer los inserts uno a uno o a través de un diccionario

A través de un for, lo añadimos a la tabla


In [45]:
# Usa este diccionario para no ir valor a valor:
datos = {
    "Luis": (24,"Madrid", 8.5),
    "Ana": (32,"Lugo", 6.25),
    "Juan": (35, "Bilbao", 5.55),
    "Nuria": (51, "Alicante", 9.75)
}

for indice,(nombre, valores) in enumerate(datos.items()): # El items, devuelve dos valores, pero al hacer el enumerate, no va a devolvernos 3, nos devolverá el índice y una tupla, si la quiero capturar he de ponerla de esta manera.
    edad = valores[0] #primer valor
    ciudad = valores[1] #segundo valor
    nota = valores[2] # tercer valor
    query = f"INSERT INTO MaSTER_cLASS (ID, NOMBRE, EDAD, CIUDAD, NOTAS) values({indice},'{nombre}', {edad}, '{ciudad}', {nota})" 
    cursor_gestion.execute(query)
    #insertamos los valores y los campos enbtre paréntesis, al introducir los numéricos {} no importa, pero el nombre, al estar pasando una cadena e interpretarla
    #tenemos que poner el nombre entre comillas, ya que el sql internamente, va a buscar el string entre comillas
    #Para ejecutarlo, debemos confirmarle al gestor que esto que hemos hecho es real, haciendo un commit

In [46]:
connection.commit() 
#con esto decimos al gestor que se guarden nlos cambios.

Para comprobar que realmente se han hecho los cambios haríamos nuestra consulta y podríamos pasarlo a un `DataFrame` pero vaya, no tenemos nuestra función, no importa porque ya es hora de que te enseñe otra cosa:

In [47]:
query = '''
SELECT * 
FROM Master_Class
'''
df = pd.read_sql(query, connection )
#no necesitamnos la función si usamos esto

In [28]:
df

,ID,NOMBRE,EDAD,CIUDAD,NOTAS
0,0,Luis,24,Madrid,8.50
1,1,Ana,32,Lugo,6.25
2,2,Juan,35,Bilbao,5.55
3,3,Nuria,51,Alicante,9.75


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      4 non-null      int64  
 1   NOMBRE  4 non-null      object 
 2   EDAD    4 non-null      int64  
 3   CIUDAD  4 non-null      object 
 4   NOTAS   4 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 288.0+ bytes


algo que no hemos hecho y que se suele hacer es cerrar las conexiones, para que las bases de datos no se queden abiertas, ya que por nuestra parte ocurre, pero por parte del gestor noesto puede generar fallos

### Update

La sintaxis general de UPDATE es:

```sql
UPDATE nombre_tabla
SET columna1 = valor1, columna2 = valor2, ...
WHERE condicion;
```

- **nombre_tabla:** Es el nombre de la tabla donde se realizarán las modificaciones.
- **SET columna1 = valor1, columna2 = valor2, ... :** Aquí se listan las columnas que se van a actualizar y los nuevos valores que se les asignarán. Puedes actualizar una o varias columnas a la vez.
- **WHERE condicion:** Especifica qué registros deben ser actualizados. La condición puede ser cualquier expresión lógica válida. Si omites la cláusula WHERE, todos los registros en la tabla serán actualizados, lo cual puede tener consecuencias no deseadas.

In [49]:
#Modifiquemos en nuestra tabla la edad de Nuria por 48 y comprobemos el resultado utlizando `read_sql`

In [50]:
query = '''
UPDATE Master_Class
SET edad = 48
WHERE nombre = "Nuria"
'''
cursor_gestion.execute(query)
pd.read_sql("SELECT * FROM Master_Class", connection)

,ID,NOMBRE,EDAD,CIUDAD,NOTAS
0,0,Luis,24,Madrid,8.50
1,1,Ana,32,Lugo,6.25
2,2,Juan,35,Bilbao,5.55
3,3,Nuria,48,Alicante,9.75


El update no fuinciona tal cual, necesitamos hacer el commit igualmente para que el cambio se grabe en la base de datos:

In [51]:
connection.commit()

In [52]:
#cerrar conexión
connection.close()

Si hubieramos intentado actualizar alguna columna incumpliendo las restricciones con las que fueron creadas hubiera dado error.

### Delete
DELETE nos permite borrar filas de las tablas, y sigue esta sintaxis:
```sql
DELETE FROM nombre_tabla WHERE condicion;
```

- **nombre_tabla:** Es el nombre de la tabla de la cual se eliminarán los registros.
- **condicion:** Especifica qué registros deben ser eliminados. Puede ser cualquier expresión lógica válida.

Si omites la cláusula WHERE en una instrucción DELETE, todos los registros de la tabla especificada serán eliminados, lo cual debe hacerse con precaución.

Borremos las filas de los alumnos con menos de un 7:

In [54]:
query = '''
DELETE FROM Master_Class'''
cursor_gestion.execute(query)

ProgrammingError: Cannot operate on a closed database.

In [55]:
pd.read_sql("SELECT * FROM Master_Class", connection) #ha borrado el contenido

ProgrammingError: Cannot operate on a closed database.

Si la queremos recuperar, debemos abrir y cerrar la base de datos para poder hacerlo:


In [35]:
connection.close()
connection = sqlite3.connect("gestion_sesion_db")
cursor_gestion = connection.cursor()

In [38]:
pd.read_sql("SELECT * FROM Master_Class", connection)

DatabaseError: Execution failed on sql 'SELECT * FROM Master_Class': no such table: Master_Class

Eliminar filas cuyas notas soin menores que 7:

In [74]:
query = '''DELETE FROM Master_Class
WHERE notas <7 '''
cursor_gestion.execute(query)
pd.read_sql("SELECT * FROM Master_Class", connection)

,ID,NOMBRE,CIUDAD,NOTAS
0,0,Luis,Madrid,8.50
1,3,Nuria,Alicante,9.75


In [73]:
#hacer el commit paraa que se quede en la base
connection.commit()

### Borrado de columnas
Igual que en pandas tenemos el método `drop` con su argumento columns para borrar columnas en SQL tenemos:


```sql
ALTER TABLE nombre_tabla DROP COLUMN nombre_columna;
```

- **nombre_tabla:** Es el nombre de la tabla de la que deseas eliminar una columna.
- **nombre_columna:** Es el nombre de la columna que deseas eliminar.



**Nota Importante:**
- Algunos sistemas de bases de datos pueden requerir modificadores adicionales o tener restricciones específicas para eliminar columnas. Por ejemplo, si otras tablas dependen de la columna que estás intentando eliminar (a través de claves foráneas, por ejemplo), es posible que necesites modificar o eliminar esas dependencias antes de poder eliminar la columna.
- Siempre es una buena práctica hacer una copia de seguridad de tu base de datos antes de realizar cambios estructurales como eliminar columnas.

 borrar la columna "edad":

In [69]:
query = '''
ALTER TABLE Master_Class
DROP COLUMN edad'''
cursor_gestion.execute(query)

OperationalError: no such column: "edad"

In [70]:
pd.read_sql("SELECT * FROM Master_Class", connection) #desaparece la columna edad

,ID,NOMBRE,CIUDAD,NOTAS
0,0,Luis,Madrid,8.50
1,3,Nuria,Alicante,9.75


In [71]:
connection.commit() #parra que se guarden los cambios

### Borrado de tablas

```sql
DROP TABLE nombre_tabla;
```

**Algunas consideraciones importantes:**

1. **Cuidado con los Datos:** Al usar `DROP TABLE`, la tabla y todos sus datos se eliminan permanentemente. No hay forma de deshacer esta acción en la mayoría de los sistemas de gestión de bases de datos. Asegúrate de que realmente deseas eliminar la tabla y de que has realizado una copia de seguridad de los datos si es necesario.

2. **Dependencias:** Si la tabla está referenciada por otras tablas a través de claves foráneas u otros mecanismos de integridad referencial, es posible que primero debas eliminar o modificar estas dependencias.

3. **Sintaxis Específica del SGBD:** La sintaxis básica de `DROP TABLE` es bastante uniforme en los diferentes sistemas de gestión de bases de datos (SGBD), pero algunos pueden ofrecer opciones adicionales. Por ejemplo, en algunos SGBD puedes usar `DROP TABLE IF EXISTS nombre_tabla;` para evitar errores si la tabla no existe.

4. **Permisos:** Necesitas tener los permisos adecuados en la base de datos para eliminar tablas. Si no tienes los permisos necesarios, la operación fallará.

In [75]:
querry = '''
DROP TABLE Master_Class
'''
cursor_gestion.execute(query)

In [76]:
pd.read_sql("SELECT Name FROM sqlite_master WHERE type = 'table'", connection)
#se borraría la tabla, pero hay que hacer commit para guardar
#hay gestores que hacen commit automático, no hay que fiarse nunca.

,name
0,Master_Class


In [77]:
#cerrar conexión
connection.close()